In [1]:
# Import required libraries and set up the path to your data
from google.colab import drive
import os
import json
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt


In [2]:
# Mount Google Drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# Path to the folder containing your data
data_dir = '/content/gdrive/MyDrive/Project 4/Combined flowers'

In [7]:
# Initialize ImageDataGenerator with augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [8]:
# Prepare the training and validation generators
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    subset='training')


Found 2508 images belonging to 10 classes.


In [9]:
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    subset='validation')

Found 621 images belonging to 10 classes.


In [10]:
# Load the MobileNetV2 model and define your own model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(train_generator.num_classes, activation='softmax'))

9406464/9406464 [==============================] - 0s 0us/step


In [13]:
# Compile the model
model.compile(optimizer=Adam(lr=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [14]:
# Define the early stop and reduce learning rate callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)


In [15]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10,
    callbacks=[early_stop, reduce_lr])

Epoch 1/10
156/156 [==============================] - 1839s 12s/step - loss: 1.1714 - accuracy: 0.6565 - val_loss: 16.1221 - val_accuracy: 0.1974 - lr: 0.0010
Epoch 2/10
156/156 [==============================] - 861s 6s/step - loss: 0.8493 - accuracy: 0.7496 - val_loss: 18.4367 - val_accuracy: 0.1102 - lr: 0.0010
Epoch 3/10
156/156 [==============================] - 851s 5s/step - loss: 0.7794 - accuracy: 0.7789 - val_loss: 12.0180 - val_accuracy: 0.2368 - lr: 0.0010
Epoch 4/10
156/156 [==============================] - 848s 5s/step - loss: 0.6502 - accuracy: 0.8194 - val_loss: 7.5665 - val_accuracy: 0.3207 - lr: 0.0010
Epoch 5/10
156/156 [==============================] - 824s 5s/step - loss: 0.5448 - accuracy: 0.8447 - val_loss: 13.8639 - val_accuracy: 0.2352 - lr: 0.0010
Epoch 6/10
  3/156 [..............................] - ETA: 12:01 - loss: 0.2627 - accuracy: 0.9167

KeyboardInterrupt: ignored